- Step 1: Set up Gmail API access 
- Step 2: Fetch your emails 
- Step 3: Convert emails to text 
- Step 4: Embed the emails 
- Step 5: Store in vector store + search

In [10]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import base64
import email

# Scopes
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

# Auth
flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)

In [ ]:
# STEP 1: Create the authorization URL
auth_url, _ = flow.authorization_url(prompt='consent')

print("Please go to this URL and authorize access:")
print(auth_url)

# STEP 2: After authorizing in the browser, Google will give you a code.
code = input("Paste the authorization code here: ")

# STEP 3: Exchange the code for credentials
flow.fetch_token(code=code)
creds = flow.credentials


Please go to this URL and authorize access:
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=85555660348-54083q32gcufsif90faqgvs4tru4d1l4.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=8gBCC8ToA6Qpbp5ckh09WpSnZXmjw3&prompt=consent&access_type=offline


In [7]:
service = build("gmail", "v1", credentials=creds)

# Fetch 10 emails
results = service.users().messages().list(userId="me", maxResults=10).execute()
messages = results.get("messages", [])

emails = []
for msg in messages:
    msg_data = service.users().messages().get(userId="me", id=msg["id"]).execute()
    payload = msg_data["payload"]
    headers = payload.get("headers", [])
    subject = next((h["value"] for h in headers if h["name"] == "Subject"), "")
    parts = payload.get("parts", [])
    
    # Get email body
    data = parts[0]["body"]["data"] if parts else payload["body"].get("data", "")
    body = base64.urlsafe_b64decode(data).decode("utf-8", errors="ignore")

    emails.append(f"Subject: {subject}\n\n{body}")

NameError: name 'creds' is not defined

In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

/home/osat/.conda/envs/osat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
